## csv 파일 읽고,  질문에 답변하기

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter 
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import FAISS 
from langchain_core.output_parsers import StrOutputParser 
from langchain_core.runnables import RunnablePassthrough 
from langchain_core.prompts import PromptTemplate 
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [9]:
# 단계 1: 문서 로드(Load Documents)
loader = CSVLoader("data/booksv_02.csv", encoding='utf-8')
docs = loader.load()
print(f"문서의 페이지수: {len(docs)}")

문서의 페이지수: 319


In [10]:
# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)
print(f"분할된 청크의수: {len(split_documents)}")

분할된 청크의수: 571


In [12]:
split_documents[1].page_content

'description: A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It?셲 1956 in Gilead, Iowa, towards the end of the Reverend Ames?셲 life, and he is absorbed in recording his family?셲 story, a legacy for the young son he will never see grow up. Haunted by his grandfather?셲 presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend?셲 lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames?셲 joyous, rambling voice that finds beauty, humour and truth in the smallest of life?셲 details, Gilead is a song of celebration and acceptance of the best and the worst

In [13]:
# 단계 3: 임베딩(Embedding) 생성
embeddings = OpenAIEmbeddings()

In [14]:
# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

In [15]:
# 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()
# 검색기에 쿼리를 날려 검색된 chunk 결과를 확인합니다.
retriever.invoke("어떤 제품의 ratings_count가 제일 높아?")

[Document(metadata={'source': 'data/booksv_02.csv', 'row': 142}, page_content='average_rating: 4.01\nnum_pages: 264\nratings_count: 8738'),
 Document(metadata={'source': 'data/booksv_02.csv', 'row': 260}, page_content='average_rating: 3.91\nnum_pages: 360\nratings_count: 34478'),
 Document(metadata={'source': 'data/booksv_02.csv', 'row': 272}, page_content='average_rating: 3.9\nnum_pages: 276\nratings_count: 1754'),
 Document(metadata={'source': 'data/booksv_02.csv', 'row': 235}, page_content='average_rating: 3.99\nnum_pages: 191\nratings_count: 1764')]

In [16]:
# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Question: 
{question} 
#Context: 
{context} 

#Answer:"""
)

In [17]:
# 단계 7: 언어모델(LLM) 생성
# 모델(LLM) 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

In [18]:
# 단계 8: 체인(Chain) 생성
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [19]:
# # 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "어떤 제품의 ratings_count가 제일 높아?"
response = chain.invoke(question)
print(response)

ratings_count가 제일 높은 제품은 ratings_count가 34478인 제품입니다.


In [26]:
question = "가장 최근에 출간된 책은?"
response = chain.invoke(question)
print(response)

가장 최근에 출간된 책은 2006년에 출간된 책입니다.


## Agent 이용

In [22]:
# csv 파일 내용 확인
import pandas as pd
df= pd.read_csv('./data/booksv_02.csv')
df.head()

,title,subtitle,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count
0,Gilead,NaN,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004,3.85,247,361
1,Spider's Web,A Novel,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000,3.83,241,5164
2,The One Tree,NaN,Stephen R. Donaldson,American fiction,http://books.google.com/books/content?id=OmQaw...,Volume Two of Stephen Donaldson's acclaimed se...,1982,3.97,479,172
3,Rage of angels,NaN,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993,3.93,512,29532
4,The Four Loves,NaN,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002,4.15,170,33684


In [23]:
# agent 설정
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.agents.agent_types import AgentType
agent = create_pandas_dataframe_agent(
    ChatOpenAI(model='gpt-4o', temperature=0),
    df,
    verbose=False,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    allow_dangerous_code=True  # 위험한 코드 실행을 허용합니다.
)

In [27]:
agent.invoke('어떤 제품의 ratings_count가 제일 높아?')

{'input': '어떤 제품의 ratings_count가 제일 높아?',
 'output': '`One Hundred Years of Solitude`가 가장 높은 ratings_count를 가지고 있습니다.'}

In [28]:
agent.invoke('가장 최근에 출간된 책은?')

{'input': '가장 최근에 출간된 책은?',
 'output': '가장 최근에 출간된 책은 "The Yiddish Policemen\'s Union"입니다.'}